Importing the necessary libraries

In [1]:
import tensorflow as tf
import os
import re
from collections import Counter
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np


Defining the necessary functions for loading the data and preprocessing the data

In [2]:
def load_data(file):
    data = []
    with open(file, 'r') as f:
        for line in f:
                line = line.strip()
                label = ' '.join(line[1:line.find("]")].strip().split())
                text = line[line.find("]")+1:].strip()
                data.append([label, text])
    return data
file = 'text.txt'
data = load_data(file)
print("Number of instances: {}".format(len(data)))
def tokenizer(token, n):
    output = []
    for i in range(len(token)-n+1):
        tokens = ' '.join(token[i:i+n])
        output.append(tokens)
    return output
def create_feature(text, nrange=(1, 1)):
    text_features = []
    text = text.lower()
    text_alphanum = re.sub('[^a-z)-9#]', ' ', text)
    for n in range(nrange[0], nrange[1]+1):
        text_features += tokenizer(text_alphanum.split(), n)
    text_punc = re.sub('[a-z0-9]', ' ', text)
    text_features += tokenizer(text_punc.split(), 1)
    return Counter(text_features)
def convert_label(item, name): 
    items = list(map(float, item.split()))
    label = ""
    for idx in range(len(items)): 
        if items[idx] == 1: 
            label += name[idx] + " "
    
    return label.strip()
emotions = ["joy", 'fear', "anger", "sadness", "disgust", "shame", "guilt"]

Number of instances: 7480


Loading the data and creating the feature vector

In [3]:
y_all = []
X_all = []
for label, text in data:
    y_all.append(convert_label(label, emotions))
    X_all.append(create_feature(text, nrange=(1, 4)))
count = {}
for i in emotions:
    count[i] = 0
for i in range(len(data)):
    count[y_all[i]] += 1
print(count)

{'joy': 1084, 'fear': 1078, 'anger': 1080, 'sadness': 1079, 'disgust': 1057, 'shame': 1045, 'guilt': 1057}


Splitting the data into training , validation and testing set

In [4]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X_all, y_all, test_size=0.3, random_state=90)
X_test, X_val, y_test, y_val = train_test_split(X_val_test, y_val_test, test_size=0.3, random_state=90)
len_Tx = len(X_train)
len_Ty = len(y_train)
len_tx = len(X_test)
len_ty = len(y_test)
len_vx = len(X_val)
len_vy = len(y_val)
print({
    "X_train": len_Tx,
    "y_train": len_Ty,
    "X_test": len_tx,
    "y_test": len_ty,
    "X_val": len_vx,
    "y_val": len_vy
})


{'X_train': 5236, 'y_train': 5236, 'X_test': 1570, 'y_test': 1570, 'X_val': 674, 'y_val': 674}


In [5]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import numpy as np

# Load pre-trained BERT model and tokenizer
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare your dataset
texts = [...]  # List of text samples
labels = [...]  # List of corresponding labels (integers)

# Tokenize texts
tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')

# Split dataset into training, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(tokenized_texts['input_ids'], labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Define TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': train_texts}, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=len(train_texts)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': val_texts}, val_labels))
val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': test_texts}, test_labels))
test_dataset = test_dataset.batch(batch_size)

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Fine-tune the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# Evaluate the model
results = model.evaluate(test_dataset)
print("Test accuracy:", results[1])


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\nikhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nikhi\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).